# Convolution and Correlation

Here we motivate and introduce the signal processing definitions of convolution and correlation. Understanding why convolution and correlation are useful for patten recognition will be key to understanding how neural networks detect patterns in data, and especially key to understanding how neural networks can detect subtle patterns in data.